In [ ]:
pip install transformers==2.6.0

In [ ]:
pip install seqeval

In [ ]:
pip install fasttext

# Baseline Model: Embedding + BiLSTM

In [ ]:
import fasttext
import numpy as np
import pandas as pd
from keras_preprocessing import sequence
from keras_preprocessing.text import Tokenizer
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('tsd_train.csv')
df.head(10)

,spans,text
0,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",Another violent and aggressive immigrant killi...
1,"[33, 34, 35, 36, 37, 38, 39]","I am 56 years old, I am not your fucking junio..."
2,"[0, 1, 2, 3]","Damn, a whole family. Sad indeed."
3,"[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]",What a knucklehead. How can anyone not know th...
4,"[32, 33, 34, 35, 36, 37, 38]","""who do you think should do the killing?""\n\nA..."
5,[],"But, but, but, is NOT a defense. It's not eve..."
6,"[39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 5...","Please people, stop using these silly, stupid ..."
7,"[0, 1, 2, 3]",Dumb.
8,"[49, 50, 51, 52, 53, 54, 147, 148, 149, 150, 1...",Obamacare is on it's last gasping breaths. Y...
9,"[32, 33, 34, 35, 36, 37, 38, 39]",CROOKED Trump = GUILTY as hell.\npathetic


In [ ]:
import re


def list_replace(search, replacement, text):
    """
    Replaces all symbols of text which are present
    in the search string with the replacement string.
    """
    search = [el for el in search if el in text]
    for c in search:
        text = text.replace(c, replacement)
    return text


def clean_text(text):
    text = list_replace('\u00AB\u00BB\u2039\u203A\u201E\u201A\u201C\u201F\u2018\u201B\u201D\u2019', '\u0022', text)
    text = list_replace('\u2012\u2013\u2014\u2015\u203E\u0305\u00AF', '\u2003\u002D\u002D\u2003', text)
    text = list_replace('\u2010\u2011', '\u002D', text)
    text = list_replace('\u2000\u2001\u2002\u2004\u2005\u2006\u2007\u2008\u2009\u200A\u200B\u202F\u205F\u2060\u3000', '\u2002', text)
    text = re.sub('\u2003\u2003', '\u2003', text)
    text = re.sub('\t\t', '\t', text)
    text = list_replace('\u02CC\u0307\u0323\u2022\u2023\u2043\u204C\u204D\u2219\u25E6\u00B7\u00D7\u22C5\u2219\u2062', '.', text)
    text = list_replace('\u2217', '\u002A', text)

    text = list_replace('\u00C4', 'A', text)
    text = list_replace('\u00E4', 'a', text)
    text = list_replace('\u00CB', 'E', text)
    text = list_replace('\u00EB', 'e', text)
    text = list_replace('\u1E26', 'H', text)
    text = list_replace('\u1E27', 'h', text)
    text = list_replace('\u00CF', 'I', text)
    text = list_replace('\u00EF', 'i', text)
    text = list_replace('\u00D6', 'O', text)
    text = list_replace('\u00F6', 'o', text)
    text = list_replace('\u00DC', 'U', text)
    text = list_replace('\u00FC', 'u', text)
    text = list_replace('\u0178', 'Y', text)
    text = list_replace('\u00FF', 'y', text)
    text = list_replace('\u00DF', 's', text)
    text = list_replace('\u1E9E', 'S', text)

    # Removing punctuation
    text = list_replace(',.[]{}()=+-−*&^%$#@!~;:§/\|\?"\n', ' ', text)
    
    # Replacing all numbers with masks
    text = list_replace('0123456789', 'x', text)

    return text

In [ ]:
# Переходим от символов к словам

X = [[]]
y = [[]]
for i in range(df.shape[0]):
    cl_text = clean_text(df.iloc[i,1])
    fl = np.zeros(len(cl_text))
    if(len(df.iloc[i,0]) > 2):
        arr = [int(a) for a in df.iloc[i,0][1:-1].split(', ')]
        fl[arr] = 1
    spl = cl_text.split(' ')
    prev_len = 0
    X_i = []
    y_i = []
    for j in range(len(spl)):
        if(len(spl[j]) > 0):
            X_i.append(spl[j])
            y_i.append(sum(fl[prev_len:prev_len+len(spl[j])])/len(spl[j]))
        prev_len = prev_len + len(spl[j]) + 1
    X.append(X_i)
    y.append(y_i)
X = X[1:]
y = y[1:]

In [ ]:
cleaned_train_texts = [' '.join(i) for i in X]
maxlen = max([len(i) for i in X])
maxlen

201

In [ ]:
! wget -O model_fasttext.bin https://www.dropbox.com/s/f8svib34687gyb4/rudrec_fasttext_model.bin?dl=0

--2020-12-20 00:01:14--  https://www.dropbox.com/s/f8svib34687gyb4/rudrec_fasttext_model.bin?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6022:18::a27d:4212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/f8svib34687gyb4/rudrec_fasttext_model.bin [following]
--2020-12-20 00:01:15--  https://www.dropbox.com/s/raw/f8svib34687gyb4/rudrec_fasttext_model.bin
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc149538eae897249b2e1ec1563e.dl.dropboxusercontent.com/cd/0/inline/BFaJGav_pdj6KOo1ne1A8EpcHG1DrSM2jOyhzsbuGAHrQXOxpf8iYXFctNDms6MtuqR0Cme_wnZ26Qu2PQMRrpkQ0HCyK_iS0249qjDC_abk_Yu__aFs4IG27Rt9Rtrwrug/file# [following]
--2020-12-20 00:01:15--  https://uc149538eae897249b2e1ec1563e.dl.dropboxusercontent.com/cd/0/inline/BFaJGav_pdj6KOo1ne1A8EpcHG1DrSM2jOyhzsbuGAHrQXOxpf8iYXFctNDms6Mt

In [ ]:
# Загружаем предобученную fasttext модель

fasttext_model_path = 'model_fasttext.bin'
fasttext_model = fasttext.load_model(fasttext_model_path)

In [ ]:
# Получаем эмбеддинги

EMBEDDINGS_DIM = 200
tokenizer = Tokenizer(lower=True, char_level=False)
tokenizer.fit_on_texts(cleaned_train_texts)
word_seq_train = tokenizer.texts_to_sequences(cleaned_train_texts)
word_index = tokenizer.word_index

trash = []
for i in range(len(X)):
    if(len(X[i]) != len(word_seq_train[i])):
        trash.append(i)
for i in trash:
    word_seq_train.pop(i)
    X.pop(i)
    y.pop(i)
  
# Дополняем нулями до длины maxlen
word_seq_train = sequence.pad_sequences(word_seq_train, maxlen=maxlen, padding="post")

dictionary_size = len(word_index.keys())
embedding_matrix = np.zeros((dictionary_size + 1, EMBEDDINGS_DIM))\

for word, i in word_index.items():
    embedding_vector = fasttext_model.get_word_vector((word))
    if (embedding_vector is not None) and len(embedding_vector) > 0:
        embedding_matrix[i] = embedding_vector

In [ ]:
# Дополняем нулями до длины maxlen

y = sequence.pad_sequences(y, maxlen=maxlen, padding="post")

In [ ]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional

# Модель со слоем эмбеддингов, двунаправленным LSTM и полносвязным слоем к каждому из выходов BiLSTM
input = Input(shape=(maxlen,))
out = Embedding(len(word_index.keys())+1, 200, input_length=maxlen, weights=[embedding_matrix])(input)
out = Dropout(0.1)(out)
out = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(out)
out = TimeDistributed(Dense(1, activation="sigmoid"))(out)
model = Model(input, out)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
from sklearn.model_selection import train_test_split

# Разбиваем выборку на обучающую и тестовую и обучаем
X_train, X_test, y_train, y_test = train_test_split(word_seq_train, y, test_size=0.2)

In [ ]:
model.fit(X_train, y_train, batch_size=32, epochs=3, validation_split=0.1, verbose=1)

Epoch 1/3
178/178 [==============================] - 213s 1s/step - loss: 0.1587 - accuracy: 0.9811 - val_loss: 0.0454 - val_accuracy: 0.9858
Epoch 2/3
178/178 [==============================] - 207s 1s/step - loss: 0.0396 - accuracy: 0.9878 - val_loss: 0.0428 - val_accuracy: 0.9873
Epoch 3/3
178/178 [==============================] - 206s 1s/step - loss: 0.0353 - accuracy: 0.9890 - val_loss: 0.0434 - val_accuracy: 0.9873


In [ ]:
pr = model.predict(X_test).reshape(y_test.shape)

In [ ]:
import tensorflow as tf

np.mean(tf.keras.losses.binary_crossentropy(y_test, pr.reshape(y_test.shape)))

0.045947444

In [ ]:
import tensorflow as tf

# Приводим выход к нужному формату для корректного подсчёта f1-score
arr1 = []
arr2 = []
for i in range(X_test.shape[0]):
    l = len(X_test[i])
    arr1.append([round(i) for i in list(pr[i][:l])])
    arr2.append(list(y_test[i][:l]))

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

print(f'Итоговый скор: {np.mean([f1_score(arr2[i], arr1[i]) for i in range(len(arr2))])}')

Итоговый скор: 0.577663187266675


# BERT Base Cased Model

## Init CUDA

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig

from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

torch.__version__

'1.7.0+cu101'

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla V100-SXM2-16GB'

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Tokenization

In [ ]:
import pandas as pd

df = pd.read_csv('tsd_train.csv')
df.head(10)

,spans,text
0,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",Another violent and aggressive immigrant killi...
1,"[33, 34, 35, 36, 37, 38, 39]","I am 56 years old, I am not your fucking junio..."
2,"[0, 1, 2, 3]","Damn, a whole family. Sad indeed."
3,"[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]",What a knucklehead. How can anyone not know th...
4,"[32, 33, 34, 35, 36, 37, 38]","""who do you think should do the killing?""\n\nA..."
5,[],"But, but, but, is NOT a defense. It's not eve..."
6,"[39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 5...","Please people, stop using these silly, stupid ..."
7,"[0, 1, 2, 3]",Dumb.
8,"[49, 50, 51, 52, 53, 54, 147, 148, 149, 150, 1...",Obamacare is on it's last gasping breaths. Y...
9,"[32, 33, 34, 35, 36, 37, 38, 39]",CROOKED Trump = GUILTY as hell.\npathetic


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)

In [ ]:
import re


def list_replace(search, replacement, text):
    """
    Replaces all symbols of text which are present
    in the search string with the replacement string.
    """
    search = [el for el in search if el in text]
    for c in search:
        text = text.replace(c, replacement)
    return text


def clean_text(text):
    """
    Cleans the given sentence
    """
    text = list_replace('\u00AB\u00BB\u2039\u203A\u201E\u201A\u201C\u201F\u2018\u201B\u201D\u2019', '\u0022', text)
    text = list_replace('\u2012\u2013\u2014\u2015\u203E\u0305\u00AF', '\u2003\u002D\u002D\u2003', text)
    text = list_replace('\u2010\u2011', '\u002D', text)
    text = list_replace('\u2000\u2001\u2002\u2004\u2005\u2006\u2007\u2008\u2009\u200A\u200B\u202F\u205F\u2060\u3000', '\u2002', text)
    text = re.sub('\u2003\u2003', '\u2003', text)
    text = re.sub('\t\t', '\t', text)
    text = list_replace('\u02CC\u0307\u0323\u2022\u2023\u2043\u204C\u204D\u2219\u25E6\u00B7\u00D7\u22C5\u2219\u2062', '.', text)
    text = list_replace('\u2217', '\u002A', text)
    text = list_replace('\u00C4', 'A', text)
    text = list_replace('\u00E4', 'a', text)
    text = list_replace('\u00CB', 'E', text)
    text = list_replace('\u00EB', 'e', text)
    text = list_replace('\u1E26', 'H', text)
    text = list_replace('\u1E27', 'h', text)
    text = list_replace('\u00CF', 'I', text)
    text = list_replace('\u00EF', 'i', text)
    text = list_replace('\u00D6', 'O', text)
    text = list_replace('\u00F6', 'o', text)
    text = list_replace('\u00DC', 'U', text)
    text = list_replace('\u00FC', 'u', text)
    text = list_replace('\u0178', 'Y', text)
    text = list_replace('\u00FF', 'y', text)
    text = list_replace('\u00DF', 's', text)
    text = list_replace('\u1E9E', 'S', text)

    # Replacing all numbers with masks
    text = list_replace('0123456789', 'x', text)

    return text

In [ ]:
df['text_normalized'] = df.text.apply(clean_text)
df.head(10)

,spans,text,text_normalized
0,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",Another violent and aggressive immigrant killi...,Another violent and aggressive immigrant killi...
1,"[33, 34, 35, 36, 37, 38, 39]","I am 56 years old, I am not your fucking junio...","I am xx years old, I am not your fucking junio..."
2,"[0, 1, 2, 3]","Damn, a whole family. Sad indeed.","Damn, a whole family. Sad indeed."
3,"[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]",What a knucklehead. How can anyone not know th...,What a knucklehead. How can anyone not know th...
4,"[32, 33, 34, 35, 36, 37, 38]","""who do you think should do the killing?""\n\nA...","""who do you think should do the killing?""\n\nA..."
5,[],"But, but, but, is NOT a defense. It's not eve...","But, but, but, is NOT a defense. It's not eve..."
6,"[39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 5...","Please people, stop using these silly, stupid ...","Please people, stop using these silly, stupid ..."
7,"[0, 1, 2, 3]",Dumb.,Dumb.
8,"[49, 50, 51, 52, 53, 54, 147, 148, 149, 150, 1...",Obamacare is on it's last gasping breaths. Y...,Obamacare is on it's last gasping breaths. Y...
9,"[32, 33, 34, 35, 36, 37, 38, 39]",CROOKED Trump = GUILTY as hell.\npathetic,CROOKED Trump = GUILTY as hell.\npathetic


In [ ]:
def create_labels(label_chars, text, embed, debug=False):
    """
    Creates tokens and labels for BERT to work with
    """
    res_tokens = []
    res_labels = []

    res_tokens.append('[CLS]')
    res_labels.append('O')

    if label_chars == '[]':
        for i in embed:
            res_tokens.append(i)
            res_labels.append('O')
    else:
        _last = -1
        _first = -1
        arr = [int(a) for a in label_chars[1:-1].split(', ')]

        bad_tokens = []
        
        for i in range(len(arr)):
            if i == 0:
                _first = arr[i]
            elif arr[i-1] + 1 != arr[i]:
                _last = arr[i-1]
                _word = tokenizer.tokenize(text[_first:_last+1])
                if debug: print(arr, _word, text)
                bad_tokens.extend(_word)
                _first = arr[i]
            elif i == len(arr)-1:
                _last = arr[i]
                _word = tokenizer.tokenize(text[_first:_last+1])
                if debug: print(arr, _word, text)
                bad_tokens.extend(_word)
        
        j = 0
        for i in embed:
            if j < len(bad_tokens) and i == bad_tokens[j]:
                j += 1
                res_tokens.append(i)
                res_labels.append('I-OFF')
            else:
                res_tokens.append(i)
                res_labels.append('O')

    res_tokens.append('[SEP]')
    res_labels.append('O')

    flag_isFirst = True
    for i in range(len(res_labels)):
        if res_labels[i] == 'I-OFF' and flag_isFirst:
            res_labels[i] = 'B-OFF'
            flag_isFirst = False
        elif res_labels[i] == 'O':
            flag_isFirst = True

    if debug:
        for i, j in zip(res_tokens, res_labels):
            print(f'{i}\t\t\t{j}')

    return res_tokens, res_labels

In [ ]:
tokenized_texts_and_labels = [
    create_labels(span, text, tokenizer.tokenize(norm)) for span, text, norm in zip(df.spans.tolist(), df.text.tolist(), df.text_normalized.tolist())
]

In [ ]:
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

In [ ]:
print(f'Длина самого большого embeding: {max(list(map(len, tokenized_texts)))}')

Длина самого большого embeding: 501


In [ ]:
MAX_LEN = 510
BATCH_SIZE = 16

In [ ]:
input_ids = pad_sequences(
    [tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
    maxlen=MAX_LEN,
    dtype="long",
    value=0.0,
    truncating="post",
    padding="post"
)

In [ ]:
tag_values = ['O', '[PAD]', 'B-OFF', 'I-OFF']
tag2idx = {t: i for i, t in enumerate(tag_values)}

In [ ]:
tags = pad_sequences(
    [[tag2idx.get(l) for l in lab] for lab in labels],
    maxlen=MAX_LEN,
    value=tag2idx["[PAD]"],
    padding="post",
    dtype="long",
    truncating="post"
)

In [ ]:
attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]

In [ ]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags, random_state=42, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids, random_state=42, test_size=0.1)

In [ ]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [ ]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=BATCH_SIZE)

## Train Model

In [ ]:
import numpy as np
import transformers

from tqdm import trange, tqdm
from seqeval.metrics import f1_score, accuracy_score
from transformers import BertForTokenClassification, AdamW, get_linear_schedule_with_warmup

transformers.__version__

'2.6.0'

In [ ]:
model = BertForTokenClassification.from_pretrained(
    "bert-base-cased",
    num_labels=len(tag2idx),
    output_attentions = False,
    output_hidden_states = False
)

In [ ]:
model.cuda();

In [ ]:
FULL_FINETUNING = True

if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {
            'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
            'weight_decay_rate': 0.01
        },
        {
            'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
            'weight_decay_rate': 0.0
        }
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=3e-5,
    eps=1e-8
)

In [ ]:
EPOCHS_NUM = 3
MAX_GRAD_NORM = 1.0

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * EPOCHS_NUM

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

In [ ]:
loss_values, validation_loss_values = [], []

for _ in trange(EPOCHS_NUM, desc="Epoch"):
    # ========================================
    #               Training
    # ========================================
    # Perform one full pass over the training set.

    # Put the model into training mode.
    model.train()
    # Reset the total loss for this epoch.
    total_loss = 0

    # Training loop
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # Always clear any previously calculated gradients before performing a backward pass.
        model.zero_grad()
        # forward pass
        # This will return the loss (rather than the model output)
        # because we have provided the `labels`.
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        # get the loss
        loss = outputs[0]
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # track train loss
        total_loss += loss.item()
        # Clip the norm of the gradient
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=MAX_GRAD_NORM)
        # update parameters
        optimizer.step()
        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)
    print("Average train loss: {}".format(avg_train_loss))

    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)


    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    # Put the model into evaluation mode
    model.eval()
    # Reset the validation loss for this epoch.
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        # Telling the model not to compute or store gradients,
        # saving memory and speeding up validation
        with torch.no_grad():
            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have not provided labels.
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        # Move logits and labels to CPU
        logits = outputs[1].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences.
        eval_loss += outputs[0].mean().item()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.extend(label_ids)

    eval_loss = eval_loss / len(valid_dataloader)
    validation_loss_values.append(eval_loss)
    print("\nValidation loss: {}".format(eval_loss))

    pred_tags = [tag_values[p_i] for p, l in zip(predictions, true_labels)
                                 for p_i, l_i in zip(p, l) if tag_values[l_i] != "[PAD]"]
    valid_tags = [tag_values[l_i] for l in true_labels
                                  for l_i in l if tag_values[l_i] != "[PAD]"]
    print("Validation Accuracy: {}".format(accuracy_score(pred_tags, valid_tags)))
    print("---")

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Average train loss: 0.240541756011429


Epoch:  33%|███▎      | 1/3 [03:51<07:42, 231.04s/it]


Validation loss: 0.21366752803325653
Validation Accuracy: 0.936051119476917
---
Average train loss: 0.19614867224322602


Epoch:  67%|██████▋   | 2/3 [07:41<03:50, 230.98s/it]


Validation loss: 0.2124370254576206
Validation Accuracy: 0.9347879928670497
---
Average train loss: 0.16086946190303605


Epoch: 100%|██████████| 3/3 [11:33<00:00, 231.11s/it]


Validation loss: 0.23559829637408256
Validation Accuracy: 0.9340945115910442
---


## Get Model Output

In [ ]:
# В качестве теста - посмотрим, что у нас получилось на одном из предложений
sent_id = 0

# Токенизация предложения
tokenized_sentence = tokenizer.encode(df.loc[sent_id].text_normalized)

# Предикт
input_ids = torch.tensor([tokenized_sentence]).cuda()
with torch.no_grad():
    output = model(input_ids)

# Приведение предикта в человеческий вид
label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)
tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])

# Вывод
print('{0: <15} {1: <5} {2: <5}'.format("TOKEN", "PRED", "TRUE"))
print("-"*27)
for i, j, k in zip(tokens, label_indices[0], labels[sent_id]):
    print('{0: <15} {1: <5} {2: <5}'.format(i, tag_values[j], k))

TOKEN           PRED  TRUE 
---------------------------
[CLS]           O     O    
Another         O     O    
violent         O     B-OFF
and             I-OFF I-OFF
aggressive      O     I-OFF
immigrant       I-OFF I-OFF
killing         I-OFF O    
a               I-OFF O    
innocent        I-OFF O    
and             O     O    
intelligent     O     O    
US              O     O    
Citizen         O     O    
.               O     O    
.               O     O    
.               O     O    
.               O     O    
Sa              O     O    
##rca           O     O    
##sm            O     O    
[SEP]           O     O    


In [ ]:
DEBUG = False

for index, row in tqdm(df.iterrows()):
    # Токенизация предложения
    tokenized_sentence = tokenizer.encode(row.text_normalized)

    # Предикт
    input_ids = torch.tensor([tokenized_sentence]).cuda()
    with torch.no_grad():
        output = model(input_ids)

    # Приведение предикта в человеческий вид
    label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)
    tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])

    # Предложение, которое мы восстанавливаем
    tmp = ''
    # Маска, в которой мы отмечаем как 0 символы не токсичного промежутка, 1 - токсичного, _ - разделители, не являющиеся токенами
    mask = ''
    # Изначальное предложение, по которому мы проверяем качество восстановления
    true = row.text

    if DEBUG: print(true)

    # Строим маску
    for token, tag in zip(tokens, label_indices[0]):
        if token in ['[CLS]', '[SEP]']:
            continue
        elif tmp == '':
            tmp += token
            if tag_values[tag] != 'O':
                mask += '1' * len(token)
            else:
                mask += '0' * len(token)
            true = true.replace(token, '', 1)
        elif not true.startswith(token):
            if token.startswith('##'):
                tmp += token[2:]
                true = true.replace(token[2:], '', 1)
                if tag_values[tag] != 'O':
                    mask += '1' * len(token[2:])
                else:
                    mask += '0' * len(token[2:])
            else:
                while not true.startswith(token) and true != '':
                    tmp += true[0]
                    true = true[1:]
                    mask += '_'
                tmp += token
                true = true.replace(token, '', 1)
                if tag_values[tag] != 'O':
                    mask += '1' * len(token)
                else:
                    mask += '0' * len(token)

    if DEBUG: print(tmp)
    if DEBUG: print(mask)

    # Восстанавливаем пробелы
    _first = -1
    _before = False
    for i in range(len(mask)):
        if mask[i] == '_' and mask[i-1]!='_':
            _first = i
            if mask[i-1] == '1':
                _before = True
            else:
                _before = False
        elif mask[i-1] == '_' and mask[i] != '_':
            if _before and mask[i] == '1':
                for j in range(_first, i):
                    mask = mask[:j] + '1' + mask[j+1:]
            else:
                for j in range(_first, i):
                    mask = mask[:j] + '0' + mask[j+1:]

    if DEBUG: print(mask)
    if DEBUG: print('---')

    # Записываем pred
    result = []
    for i in range(len(mask)):
        if mask[i] == '1':
            result.append(i)
    df.at[index, 'pred'] = str(result)

7939it [01:49, 72.38it/s]


In [ ]:
for index, row in tqdm(df.iterrows()):
    if row.spans == '[]' and row.pred == '[]':
        df.at[index, 'score'] = 1
    elif row.spans == '[]' and row.pred != '[]':
        df.at[index, 'score'] = 0
    elif row.spans != '[]' and row.pred == '[]':
        df.at[index, 'score'] = 0
    else:
        arr_a = set([int(a) for a in row.pred[1:-1].split(', ')])
        arr_g = set([int(a) for a in row.spans[1:-1].split(', ')])

        p = len(arr_a & arr_g) / len(arr_a)
        r = len(arr_a & arr_g) / len(arr_g)

        df.at[index, 'score'] = (2 * p * r) / (p + r) if (p + r) != 0 else 0

7939it [00:01, 5755.06it/s]


In [ ]:
print(f'Итоговый скор: {df.score.mean()}')

Итоговый скор: 0.6892408996963809


# BERT Lagre Cased Model

## Init CUDA

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig

from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

torch.__version__

'1.7.0+cu101'

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla V100-SXM2-16GB'

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Tokenization

In [ ]:
import pandas as pd

df = pd.read_csv('tsd_train.csv')
df.head(10)

,spans,text
0,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",Another violent and aggressive immigrant killi...
1,"[33, 34, 35, 36, 37, 38, 39]","I am 56 years old, I am not your fucking junio..."
2,"[0, 1, 2, 3]","Damn, a whole family. Sad indeed."
3,"[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]",What a knucklehead. How can anyone not know th...
4,"[32, 33, 34, 35, 36, 37, 38]","""who do you think should do the killing?""\n\nA..."
5,[],"But, but, but, is NOT a defense. It's not eve..."
6,"[39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 5...","Please people, stop using these silly, stupid ..."
7,"[0, 1, 2, 3]",Dumb.
8,"[49, 50, 51, 52, 53, 54, 147, 148, 149, 150, 1...",Obamacare is on it's last gasping breaths. Y...
9,"[32, 33, 34, 35, 36, 37, 38, 39]",CROOKED Trump = GUILTY as hell.\npathetic


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-large-cased', do_lower_case=False)

In [ ]:
import re


def list_replace(search, replacement, text):
    """
    Replaces all symbols of text which are present
    in the search string with the replacement string.
    """
    search = [el for el in search if el in text]
    for c in search:
        text = text.replace(c, replacement)
    return text


def clean_text(text):
    """
    Cleans the given sentence
    """
    text = list_replace('\u00AB\u00BB\u2039\u203A\u201E\u201A\u201C\u201F\u2018\u201B\u201D\u2019', '\u0022', text)
    text = list_replace('\u2012\u2013\u2014\u2015\u203E\u0305\u00AF', '\u2003\u002D\u002D\u2003', text)
    text = list_replace('\u2010\u2011', '\u002D', text)
    text = list_replace('\u2000\u2001\u2002\u2004\u2005\u2006\u2007\u2008\u2009\u200A\u200B\u202F\u205F\u2060\u3000', '\u2002', text)
    text = re.sub('\u2003\u2003', '\u2003', text)
    text = re.sub('\t\t', '\t', text)
    text = list_replace('\u02CC\u0307\u0323\u2022\u2023\u2043\u204C\u204D\u2219\u25E6\u00B7\u00D7\u22C5\u2219\u2062', '.', text)
    text = list_replace('\u2217', '\u002A', text)
    text = list_replace('\u00C4', 'A', text)
    text = list_replace('\u00E4', 'a', text)
    text = list_replace('\u00CB', 'E', text)
    text = list_replace('\u00EB', 'e', text)
    text = list_replace('\u1E26', 'H', text)
    text = list_replace('\u1E27', 'h', text)
    text = list_replace('\u00CF', 'I', text)
    text = list_replace('\u00EF', 'i', text)
    text = list_replace('\u00D6', 'O', text)
    text = list_replace('\u00F6', 'o', text)
    text = list_replace('\u00DC', 'U', text)
    text = list_replace('\u00FC', 'u', text)
    text = list_replace('\u0178', 'Y', text)
    text = list_replace('\u00FF', 'y', text)
    text = list_replace('\u00DF', 's', text)
    text = list_replace('\u1E9E', 'S', text)

    # Replacing all numbers with masks
    text = list_replace('0123456789', 'x', text)

    return text

In [ ]:
df['text_normalized'] = df.text.apply(clean_text)
df.head(10)

,spans,text,text_normalized
0,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",Another violent and aggressive immigrant killi...,Another violent and aggressive immigrant killi...
1,"[33, 34, 35, 36, 37, 38, 39]","I am 56 years old, I am not your fucking junio...","I am xx years old, I am not your fucking junio..."
2,"[0, 1, 2, 3]","Damn, a whole family. Sad indeed.","Damn, a whole family. Sad indeed."
3,"[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]",What a knucklehead. How can anyone not know th...,What a knucklehead. How can anyone not know th...
4,"[32, 33, 34, 35, 36, 37, 38]","""who do you think should do the killing?""\n\nA...","""who do you think should do the killing?""\n\nA..."
5,[],"But, but, but, is NOT a defense. It's not eve...","But, but, but, is NOT a defense. It's not eve..."
6,"[39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 5...","Please people, stop using these silly, stupid ...","Please people, stop using these silly, stupid ..."
7,"[0, 1, 2, 3]",Dumb.,Dumb.
8,"[49, 50, 51, 52, 53, 54, 147, 148, 149, 150, 1...",Obamacare is on it's last gasping breaths. Y...,Obamacare is on it's last gasping breaths. Y...
9,"[32, 33, 34, 35, 36, 37, 38, 39]",CROOKED Trump = GUILTY as hell.\npathetic,CROOKED Trump = GUILTY as hell.\npathetic


In [ ]:
def create_labels(label_chars, text, embed, debug=False):
    """
    Creates tokens and labels for BERT to work with
    """
    res_tokens = []
    res_labels = []

    res_tokens.append('[CLS]')
    res_labels.append('O')

    if label_chars == '[]':
        for i in embed:
            res_tokens.append(i)
            res_labels.append('O')
    else:
        _last = -1
        _first = -1
        arr = [int(a) for a in label_chars[1:-1].split(', ')]

        bad_tokens = []
        
        for i in range(len(arr)):
            if i == 0:
                _first = arr[i]
            elif arr[i-1] + 1 != arr[i]:
                _last = arr[i-1]
                _word = tokenizer.tokenize(text[_first:_last+1])
                if debug: print(arr, _word, text)
                bad_tokens.extend(_word)
                _first = arr[i]
            elif i == len(arr)-1:
                _last = arr[i]
                _word = tokenizer.tokenize(text[_first:_last+1])
                if debug: print(arr, _word, text)
                bad_tokens.extend(_word)
        
        j = 0
        for i in embed:
            if j < len(bad_tokens) and i == bad_tokens[j]:
                j += 1
                res_tokens.append(i)
                res_labels.append('I-OFF')
            else:
                res_tokens.append(i)
                res_labels.append('O')

    res_tokens.append('[SEP]')
    res_labels.append('O')

    flag_isFirst = True
    for i in range(len(res_labels)):
        if res_labels[i] == 'I-OFF' and flag_isFirst:
            res_labels[i] = 'B-OFF'
            flag_isFirst = False
        elif res_labels[i] == 'O':
            flag_isFirst = True

    if debug:
        for i, j in zip(res_tokens, res_labels):
            print(f'{i}\t\t\t{j}')

    return res_tokens, res_labels

In [ ]:
tokenized_texts_and_labels = [
    create_labels(span, text, tokenizer.tokenize(norm)) for span, text, norm in zip(df.spans.tolist(), df.text.tolist(), df.text_normalized.tolist())
]

In [ ]:
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

In [ ]:
print(f'Длина самого большого embeding: {max(list(map(len, tokenized_texts)))}')

Длина самого большого embeding: 501


In [ ]:
MAX_LEN = 510
BATCH_SIZE = 4

In [ ]:
input_ids = pad_sequences(
    [tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
    maxlen=MAX_LEN,
    dtype="long",
    value=0.0,
    truncating="post",
    padding="post"
)

In [ ]:
tag_values = ['O', '[PAD]', 'B-OFF', 'I-OFF']
tag2idx = {t: i for i, t in enumerate(tag_values)}

In [ ]:
tags = pad_sequences(
    [[tag2idx.get(l) for l in lab] for lab in labels],
    maxlen=MAX_LEN,
    value=tag2idx["[PAD]"],
    padding="post",
    dtype="long",
    truncating="post"
)

In [ ]:
attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]

In [ ]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags, random_state=42, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids, random_state=42, test_size=0.1)

In [ ]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [ ]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=BATCH_SIZE)

## Train Model

In [ ]:
import numpy as np
import transformers

from tqdm import trange, tqdm
from seqeval.metrics import f1_score, accuracy_score
from transformers import BertForTokenClassification, AdamW, get_linear_schedule_with_warmup

transformers.__version__

'2.6.0'

In [ ]:
model = BertForTokenClassification.from_pretrained(
    "bert-large-cased",
    num_labels=len(tag2idx),
    output_attentions = False,
    output_hidden_states = False
)

In [ ]:
model.cuda();

In [ ]:
FULL_FINETUNING = True

if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {
            'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
            'weight_decay_rate': 0.01
        },
        {
            'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
            'weight_decay_rate': 0.0
        }
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=3e-5,
    eps=1e-8
)

In [ ]:
EPOCHS_NUM = 3
MAX_GRAD_NORM = 1.0

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * EPOCHS_NUM

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

In [ ]:
loss_values, validation_loss_values = [], []

for _ in trange(EPOCHS_NUM, desc="Epoch"):
    # ========================================
    #               Training
    # ========================================
    # Perform one full pass over the training set.

    # Put the model into training mode.
    model.train()
    # Reset the total loss for this epoch.
    total_loss = 0

    # Training loop
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # Always clear any previously calculated gradients before performing a backward pass.
        model.zero_grad()
        # forward pass
        # This will return the loss (rather than the model output)
        # because we have provided the `labels`.
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        # get the loss
        loss = outputs[0]
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # track train loss
        total_loss += loss.item()
        # Clip the norm of the gradient
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=MAX_GRAD_NORM)
        # update parameters
        optimizer.step()
        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)
    print("Average train loss: {}".format(avg_train_loss))

    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)


    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    # Put the model into evaluation mode
    model.eval()
    # Reset the validation loss for this epoch.
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        # Telling the model not to compute or store gradients,
        # saving memory and speeding up validation
        with torch.no_grad():
            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have not provided labels.
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        # Move logits and labels to CPU
        logits = outputs[1].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences.
        eval_loss += outputs[0].mean().item()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.extend(label_ids)

    eval_loss = eval_loss / len(valid_dataloader)
    validation_loss_values.append(eval_loss)
    print("\nValidation loss: {}".format(eval_loss))

    pred_tags = [tag_values[p_i] for p, l in zip(predictions, true_labels)
                                 for p_i, l_i in zip(p, l) if tag_values[l_i] != "[PAD]"]
    valid_tags = [tag_values[l_i] for l in true_labels
                                  for l_i in l if tag_values[l_i] != "[PAD]"]
    print("Validation Accuracy: {}".format(accuracy_score(pred_tags, valid_tags)))
    print("---")

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Average train loss: 0.24870469009792448


Epoch:  33%|███▎      | 1/3 [14:22<28:45, 862.88s/it]


Validation loss: 0.23953121398861085
Validation Accuracy: 0.9360758866653458
---
Average train loss: 0.20286594124596355


Epoch:  67%|██████▋   | 2/3 [28:45<14:22, 862.69s/it]


Validation loss: 0.2350809817768671
Validation Accuracy: 0.9371408757677828
---
Average train loss: 0.13105480274744227


Epoch: 100%|██████████| 3/3 [43:07<00:00, 862.59s/it]


Validation loss: 0.3371578547050568
Validation Accuracy: 0.9293144442242917
---


## Get Model Output

In [ ]:
# В качестве теста - посмотрим, что у нас получилось на одном из предложений
sent_id = 0

# Токенизация предложения
tokenized_sentence = tokenizer.encode(df.loc[sent_id].text_normalized)

# Предикт
input_ids = torch.tensor([tokenized_sentence]).cuda()
with torch.no_grad():
    output = model(input_ids)

# Приведение предикта в человеческий вид
label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)
tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])

# Вывод
print('{0: <15} {1: <5} {2: <5}'.format("TOKEN", "PRED", "TRUE"))
print("-"*27)
for i, j, k in zip(tokens, label_indices[0], labels[sent_id]):
    print('{0: <15} {1: <5} {2: <5}'.format(i, tag_values[j], k))

TOKEN           PRED  TRUE 
---------------------------
[CLS]           O     O    
Another         O     O    
violent         B-OFF B-OFF
and             I-OFF I-OFF
aggressive      I-OFF I-OFF
immigrant       I-OFF I-OFF
killing         O     O    
a               O     O    
innocent        O     O    
and             O     O    
intelligent     O     O    
US              O     O    
Citizen         O     O    
.               O     O    
.               O     O    
.               O     O    
.               O     O    
Sa              O     O    
##rca           O     O    
##sm            O     O    
[SEP]           O     O    


In [ ]:
DEBUG = False

for index, row in tqdm(df.iterrows()):
    # Токенизация предложения
    tokenized_sentence = tokenizer.encode(row.text_normalized)

    # Предикт
    input_ids = torch.tensor([tokenized_sentence]).cuda()
    with torch.no_grad():
        output = model(input_ids)

    # Приведение предикта в человеческий вид
    label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)
    tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])

    # Предложение, которое мы восстанавливаем
    tmp = ''
    # Маска, в которой мы отмечаем как 0 символы не токсичного промежутка, 1 - токсичного, _ - разделители, не являющиеся токенами
    mask = ''
    # Изначальное предложение, по которому мы проверяем качество восстановления
    true = row.text

    if DEBUG: print(true)

    # Строим маску
    for token, tag in zip(tokens, label_indices[0]):
        if token in ['[CLS]', '[SEP]']:
            continue
        elif tmp == '':
            tmp += token
            if tag_values[tag] != 'O':
                mask += '1' * len(token)
            else:
                mask += '0' * len(token)
            true = true.replace(token, '', 1)
        elif not true.startswith(token):
            if token.startswith('##'):
                tmp += token[2:]
                true = true.replace(token[2:], '', 1)
                if tag_values[tag] != 'O':
                    mask += '1' * len(token[2:])
                else:
                    mask += '0' * len(token[2:])
            else:
                while not true.startswith(token) and true != '':
                    tmp += true[0]
                    true = true[1:]
                    mask += '_'
                tmp += token
                true = true.replace(token, '', 1)
                if tag_values[tag] != 'O':
                    mask += '1' * len(token)
                else:
                    mask += '0' * len(token)

    if DEBUG: print(tmp)
    if DEBUG: print(mask)

    # Восстанавливаем пробелы
    _first = -1
    _before = False
    for i in range(len(mask)):
        if mask[i] == '_' and mask[i-1]!='_':
            _first = i
            if mask[i-1] == '1':
                _before = True
            else:
                _before = False
        elif mask[i-1] == '_' and mask[i] != '_':
            if _before and mask[i] == '1':
                for j in range(_first, i):
                    mask = mask[:j] + '1' + mask[j+1:]
            else:
                for j in range(_first, i):
                    mask = mask[:j] + '0' + mask[j+1:]

    if DEBUG: print(mask)
    if DEBUG: print('---')

    # Записываем pred
    result = []
    for i in range(len(mask)):
        if mask[i] == '1':
            result.append(i)
    df.at[index, 'pred'] = str(result)

7939it [03:20, 39.63it/s]


In [ ]:
for index, row in tqdm(df.iterrows()):
    if row.spans == '[]' and row.pred == '[]':
        df.at[index, 'score'] = 1
    elif row.spans == '[]' and row.pred != '[]':
        df.at[index, 'score'] = 0
    elif row.spans != '[]' and row.pred == '[]':
        df.at[index, 'score'] = 0
    else:
        arr_a = set([int(a) for a in row.pred[1:-1].split(', ')])
        arr_g = set([int(a) for a in row.spans[1:-1].split(', ')])

        p = len(arr_a & arr_g) / len(arr_a)
        r = len(arr_a & arr_g) / len(arr_g)

        df.at[index, 'score'] = (2 * p * r) / (p + r) if (p + r) != 0 else 0

7939it [00:01, 5855.42it/s]


In [ ]:
print(f'Итоговый скор: {df.score.mean()}')

Итоговый скор: 0.7626692466158488
